In [1]:
import pandas as pd
df = pd.read_csv('/mnt/DAE855F7E855D1FD/github_msc/NLP/HW3/train_tag.csv')
df.head()

,word,tag
0,ميشوند,V
1,.,DELM
2,,<S>
3,نتيجهي,N
4,بحث,N


In [2]:
df.shape

(268515, 2)

In [3]:
new_row = pd.DataFrame({'word':" ", 'tag': "<S>"}, index =[0])
df = pd.concat([new_row, df]).reset_index(drop = True)
df.head()

,word,tag
0,,<S>
1,ميشوند,V
2,.,DELM
3,,<S>
4,نتيجهي,N


In [4]:
df.shape

(268516, 2)

In [5]:
word_count = df.word.value_counts().to_dict()
word_count

{'و': 11775,
 ' ': 8722,
 '.': 8722,
 'در': 8471,
 'به': 7206,
 '،': 7158,
 'از': 6163,
 'كه': 5507,
 'است': 4210,
 'اين': 3708,
 'را': 3495,
 'با': 2671,
 '-': 1745,
 '(': 1694,
 ':': 1690,
 'آن': 1607,
 ')': 1436,
 'براي': 1350,
 'يك': 1253,
 'بر': 1176,
 'ميشود': 1017,
 '#': 884,
 'خود': 828,
 'يا': 794,
 'هر': 746,
 'آنها': 738,
 'شده': 725,
 'مورد': 703,
 '"': 700,
 'تا': 694,
 'نيز': 680,
 'شد': 627,
 'دو': 610,
 'قرار': 604,
 'بود': 601,
 'او': 586,
 'دارد': 585,
 'صورت': 566,
 'نظر': 562,
 'سال': 556,
 'باشد': 550,
 '2': 524,
 'ديگر': 499,
 'كار': 492,
 'هم': 491,
 'گروه': 487,
 '1': 472,
 'ميكند': 464,
 'وجود': 450,
 'بين': 441,
 'بايد': 435,
 'استفاده': 434,
 'آموزش': 431,
 '/': 427,
 'كرد': 415,
 'ميباشد': 404,
 'توجه': 380,
 'زيروبمي': 380,
 'شود': 373,
 'انجام': 368,
 'آب': 358,
 'اگر': 356,
 'بعد': 355,
 'هستند': 349,
 '3': 347,
 'نشان': 334,
 'يعني': 319,
 'عنوان': 313,
 'طور': 311,
 'روي': 310,
 'سطح': 296,
 'درصد': 289,
 'اجتماعي': 289,
 'ولي': 288,
 'نفت': 287,
 'زير'

In [6]:
len(word_count)

18950

In [7]:
tag_count = df.tag.value_counts().to_dict()
tag_count

{'N': 109310,
 'P': 32580,
 'ADJ': 29832,
 'DELM': 25844,
 'V': 22327,
 'CON': 22094,
 '<S>': 8722,
 'PRO': 5714,
 'DET': 4130,
 'ADV': 3646,
 'QUA': 1820,
 'AR': 1175,
 'IF': 396,
 'SPEC': 350,
 'MS': 198,
 'MORP': 132,
 'PP': 85,
 'MQUA': 81,
 'PS': 31,
 'DEFAULT': 20,
 'OH': 12,
 'NP': 10,
 'OHH': 5,
 'INT': 2}

In [8]:
len(tag_count)

24

In [9]:
tag_seq = df.tag.to_list()
len(tag_seq)

268516

In [12]:
import numpy as np
def create_transition_matrix(df, alpha): # df : tag_word_dataframe
    # transiont_count dictionary
    tag_count = df.tag.value_counts().to_dict()
    all_tags = sorted(tag_count.keys())
    tag_seq = df.tag.to_list()
    transition_count = {} 
    for i in all_tags:
        for j in all_tags:
            key = (i, j)
            transition_count[key] = 0

    for i in range(len(tag_seq)-1):
        transition_count[(tag_seq[i], tag_seq[i+1])] += 1

    # create transition matrix : bigram
    transition_matrix = np.zeros((len(all_tags), len(all_tags)))

    for i in range(transition_matrix.shape[0]):
        for j in range(transition_matrix.shape[1]):
            count = 0
            key = (all_tags[i], all_tags[j])
            if key in transition_count:
                count = transition_count[key]
            count_prev_tag = tag_count[all_tags[i]]
            transition_matrix[i][j] = (count + alpha) / (count_prev_tag + alpha* len(all_tags))
    transition_matrix = np.delete(transition_matrix, 0, axis=1) # eliminate column 1 correspondes to <S> tag       
    return transition_matrix        

In [13]:
x = create_transition_matrix(df)
x.shape

(24, 24)

In [15]:
transition_count

{('<S>', '<S>'): 0,
 ('<S>', 'ADJ'): 184,
 ('<S>', 'ADV'): 339,
 ('<S>', 'AR'): 3,
 ('<S>', 'CON'): 911,
 ('<S>', 'DEFAULT'): 13,
 ('<S>', 'DELM'): 716,
 ('<S>', 'DET'): 382,
 ('<S>', 'IF'): 106,
 ('<S>', 'INT'): 1,
 ('<S>', 'MORP'): 0,
 ('<S>', 'MQUA'): 3,
 ('<S>', 'MS'): 30,
 ('<S>', 'N'): 3979,
 ('<S>', 'NP'): 0,
 ('<S>', 'OH'): 0,
 ('<S>', 'OHH'): 1,
 ('<S>', 'P'): 1531,
 ('<S>', 'PP'): 8,
 ('<S>', 'PRO'): 345,
 ('<S>', 'PS'): 2,
 ('<S>', 'QUA'): 97,
 ('<S>', 'SPEC'): 9,
 ('<S>', 'V'): 62,
 ('ADJ', '<S>'): 0,
 ('ADJ', 'ADJ'): 2565,
 ('ADJ', 'ADV'): 350,
 ('ADJ', 'AR'): 0,
 ('ADJ', 'CON'): 3448,
 ('ADJ', 'DEFAULT'): 1,
 ('ADJ', 'DELM'): 2412,
 ('ADJ', 'DET'): 197,
 ('ADJ', 'IF'): 15,
 ('ADJ', 'INT'): 0,
 ('ADJ', 'MORP'): 9,
 ('ADJ', 'MQUA'): 10,
 ('ADJ', 'MS'): 3,
 ('ADJ', 'N'): 7560,
 ('ADJ', 'NP'): 0,
 ('ADJ', 'OH'): 0,
 ('ADJ', 'OHH'): 0,
 ('ADJ', 'P'): 5654,
 ('ADJ', 'PP'): 6,
 ('ADJ', 'PRO'): 531,
 ('ADJ', 'PS'): 0,
 ('ADJ', 'QUA'): 106,
 ('ADJ', 'SPEC'): 12,
 ('ADJ', 'V'): 695

In [20]:
import numpy as np

unique_tags = len(tag_count)  # except <S> tag
# unique_words = len(word_count)  # except empty word
all_tags = sorted(tag_count.keys())

# transition_matrix = np.zeros((unique_tags, unique_tags))
# # transition_count = {()} 
# # create transition matrix
# for i in range(transition_matrix.shape[0]):
#     for j in range(transition_matrix.shape[1]):
#         count = 0
#         key = (all_tags[i], all_tags[j])
#         if key in transition_count:
#             count = transition_count[key]
#         count_prev_tag = tag_count[all_tags[i]]
#         transition_matrix[i][j] = count / count_prev_tag

In [17]:
all_tags[0]

'<S>'

In [18]:
transition_matrix

array([[0.00000000e+00, 2.10960789e-02, 3.88672323e-02, 3.43957808e-04,
        1.04448521e-01, 1.49048383e-03, 8.20912635e-02, 4.37972942e-02,
        1.21531759e-02, 1.14652603e-04, 0.00000000e+00, 3.43957808e-04,
        3.43957808e-03, 4.56202706e-01, 0.00000000e+00, 0.00000000e+00,
        1.14652603e-04, 1.75533135e-01, 9.17220821e-04, 3.95551479e-02,
        2.29305205e-04, 1.11213025e-02, 1.03187342e-03, 7.10846136e-03],
       [0.00000000e+00, 8.59814964e-02, 1.17323679e-02, 0.00000000e+00,
        1.15580585e-01, 3.35210512e-05, 8.08527755e-02, 6.60364709e-03,
        5.02815768e-04, 0.00000000e+00, 3.01689461e-04, 3.35210512e-04,
        1.00563154e-04, 2.53419147e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.89528024e-01, 2.01126307e-04, 1.77996782e-02,
        0.00000000e+00, 3.55323143e-03, 4.02252615e-04, 2.33071869e-01],
       [0.00000000e+00, 1.72517828e-01, 2.24904004e-02, 0.00000000e+00,
        2.71530444e-02, 0.00000000e+00, 4.11409764e-02, 1.8924

In [90]:
# np.save('/mnt/DAE855F7E855D1FD/github_msc/NLP/HW3/transition_matrix.npy', transition_matrix)

In [89]:
transition_matrix.shape

(24, 24)

In [24]:
x = np.load('/mnt/DAE855F7E855D1FD/github_msc/NLP/HW3/transition_matrix.npy')
x.shape

(24, 24)

In [25]:
x = np.delete(x, 0, axis=1)
x.shape

(24, 23)

In [22]:
all_tags1 = [tag for tag in all_tags if tag != "<S>"]
len(all_tags1)

23

In [23]:
all_tags1

['ADJ',
 'ADV',
 'AR',
 'CON',
 'DEFAULT',
 'DELM',
 'DET',
 'IF',
 'INT',
 'MORP',
 'MQUA',
 'MS',
 'N',
 'NP',
 'OH',
 'OHH',
 'P',
 'PP',
 'PRO',
 'PS',
 'QUA',
 'SPEC',
 'V']

In [26]:
tags_df1 = pd.DataFrame(x, columns = list(all_tags1), index=list(all_tags))
display(tags_df1)

,ADJ,ADV,AR,CON,DEFAULT,DELM,DET,IF,INT,MORP,...,NP,OH,OHH,P,PP,PRO,PS,QUA,SPEC,V
<S>,0.021096,0.038867,0.000344,0.104449,0.001490,0.082091,0.043797,0.012153,0.000115,0.000000,...,0.000000,0.000000,0.000115,0.175533,0.000917,0.039555,0.000229,0.011121,0.001032,0.007108
ADJ,0.085981,0.011732,0.000000,0.115581,0.000034,0.080853,0.006604,0.000503,0.000000,0.000302,...,0.000000,0.000000,0.000000,0.189528,0.000201,0.017800,0.000000,0.003553,0.000402,0.233072
ADV,0.172518,0.022490,0.000000,0.027153,0.000000,0.041141,0.018925,0.005485,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.213933,0.000274,0.024136,0.000000,0.007131,0.000549,0.065551
AR,0.001702,0.000000,0.797447,0.012766,0.000000,0.175319,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000851,0.000000,0.000851,0.000851,0.000000,0.000000,0.001702
CON,0.093012,0.038744,0.000407,0.039966,0.000000,0.012311,0.024305,0.006427,0.000000,0.000000,...,0.000045,0.000091,0.000045,0.183987,0.000498,0.024803,0.000136,0.017742,0.000996,0.026885
DEFAULT,0.000000,0.050000,0.000000,0.050000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.050000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000
DELM,0.033315,0.018341,0.007700,0.057924,0.000077,0.043182,0.008513,0.002554,0.000039,0.000039,...,0.000000,0.000271,0.000077,0.061407,0.000310,0.009325,0.000929,0.004721,0.000310,0.013581
DET,0.002906,0.000000,0.000000,0.012833,0.000000,0.000969,0.000242,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.002906,0.001453,0.001211,0.000000,0.000242,0.019128,0.008717
IF,0.055556,0.020202,0.000000,0.010101,0.000000,0.002525,0.042929,0.002525,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.136364,0.000000,0.025253,0.000000,0.022727,0.002525,0.060606
INT,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
tags_df = pd.DataFrame(transition_matrix, columns = list(all_tags), index=list(all_tags))
display(tags_df)

,<S>,ADJ,ADV,AR,CON,DEFAULT,DELM,DET,IF,INT,...,NP,OH,OHH,P,PP,PRO,PS,QUA,SPEC,V
<S>,0.000000,0.021096,0.038867,0.000344,0.104449,0.001490,0.082091,0.043797,0.012153,0.000115,...,0.000000,0.000000,0.000115,0.175533,0.000917,0.039555,0.000229,0.011121,0.001032,0.007108
ADJ,0.000000,0.085981,0.011732,0.000000,0.115581,0.000034,0.080853,0.006604,0.000503,0.000000,...,0.000000,0.000000,0.000000,0.189528,0.000201,0.017800,0.000000,0.003553,0.000402,0.233072
ADV,0.000000,0.172518,0.022490,0.000000,0.027153,0.000000,0.041141,0.018925,0.005485,0.000000,...,0.000000,0.000000,0.000000,0.213933,0.000274,0.024136,0.000000,0.007131,0.000549,0.065551
AR,0.000000,0.001702,0.000000,0.797447,0.012766,0.000000,0.175319,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000851,0.000000,0.000851,0.000851,0.000000,0.000000,0.001702
CON,0.000000,0.093012,0.038744,0.000407,0.039966,0.000000,0.012311,0.024305,0.006427,0.000000,...,0.000045,0.000091,0.000045,0.183987,0.000498,0.024803,0.000136,0.017742,0.000996,0.026885
DEFAULT,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.100000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.050000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000
DELM,0.337448,0.033315,0.018341,0.007700,0.057924,0.000077,0.043182,0.008513,0.002554,0.000039,...,0.000000,0.000271,0.000077,0.061407,0.000310,0.009325,0.000929,0.004721,0.000310,0.013581
DET,0.000000,0.002906,0.000000,0.000000,0.012833,0.000000,0.000969,0.000242,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.002906,0.001453,0.001211,0.000000,0.000242,0.019128,0.008717
IF,0.000000,0.055556,0.020202,0.000000,0.010101,0.000000,0.002525,0.042929,0.002525,0.000000,...,0.000000,0.000000,0.000000,0.136364,0.000000,0.025253,0.000000,0.022727,0.002525,0.060606
INT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
import operator
sorted_d = dict( sorted(word_count.items(), key=operator.itemgetter(1),reverse=True))

In [31]:
import operator

def extract_vocab(df):
    word_count = df.word.value_counts().to_dict()
    sorted_word_count = dict(sorted(word_count.items(), key=operator.itemgetter(1),reverse=True))
    del sorted_word_count[" "] # delete blanck lines
    
    single_occurrence_words = [] # threshold
    for word in sorted_word_count:
        if sorted_word_count[word] == 1:
            single_occurrence_words.append(word)
    new_word_counts = {k:v for k,v in sorted_word_count.items() if v != 1}
    new_word_counts["UNK"] = len(single_occurrence_words)
    new_word_counts = dict(sorted(new_word_counts.items(), key=operator.itemgetter(1),reverse=True))
    return new_word_counts # keys are our vocabulary

In [23]:
sorted_d

{'و': 11775,
 ' ': 8722,
 '.': 8722,
 'در': 8471,
 'به': 7206,
 '،': 7158,
 'از': 6163,
 'كه': 5507,
 'است': 4210,
 'اين': 3708,
 'را': 3495,
 'با': 2671,
 '-': 1745,
 '(': 1694,
 ':': 1690,
 'آن': 1607,
 ')': 1436,
 'براي': 1350,
 'يك': 1253,
 'بر': 1176,
 'ميشود': 1017,
 '#': 884,
 'خود': 828,
 'يا': 794,
 'هر': 746,
 'آنها': 738,
 'شده': 725,
 'مورد': 703,
 '"': 700,
 'تا': 694,
 'نيز': 680,
 'شد': 627,
 'دو': 610,
 'قرار': 604,
 'بود': 601,
 'او': 586,
 'دارد': 585,
 'صورت': 566,
 'نظر': 562,
 'سال': 556,
 'باشد': 550,
 '2': 524,
 'ديگر': 499,
 'كار': 492,
 'هم': 491,
 'گروه': 487,
 '1': 472,
 'ميكند': 464,
 'وجود': 450,
 'بين': 441,
 'بايد': 435,
 'استفاده': 434,
 'آموزش': 431,
 '/': 427,
 'كرد': 415,
 'ميباشد': 404,
 'توجه': 380,
 'زيروبمي': 380,
 'شود': 373,
 'انجام': 368,
 'آب': 358,
 'اگر': 356,
 'بعد': 355,
 'هستند': 349,
 '3': 347,
 'نشان': 334,
 'يعني': 319,
 'عنوان': 313,
 'طور': 311,
 'روي': 310,
 'سطح': 296,
 'درصد': 289,
 'اجتماعي': 289,
 'ولي': 288,
 'نفت': 287,
 'زير'

In [24]:
del sorted_d[" "]

In [25]:
len(sorted_d)

18949

In [28]:
single_occurrence_words = []
for i in sorted_d:
    if sorted_d[i] == 1:
        single_occurrence_words.append(i)

In [29]:
len(single_occurrence_words)

8732

In [34]:
new_word_count = {k:v for k,v in sorted_d.items() if v != 1}

In [35]:
len(new_word_count)

10217

In [36]:
new_word_count["UNK"] = len(single_occurrence_words)

In [38]:
new_word_count

{'و': 11775,
 '.': 8722,
 'در': 8471,
 'به': 7206,
 '،': 7158,
 'از': 6163,
 'كه': 5507,
 'است': 4210,
 'اين': 3708,
 'را': 3495,
 'با': 2671,
 '-': 1745,
 '(': 1694,
 ':': 1690,
 'آن': 1607,
 ')': 1436,
 'براي': 1350,
 'يك': 1253,
 'بر': 1176,
 'ميشود': 1017,
 '#': 884,
 'خود': 828,
 'يا': 794,
 'هر': 746,
 'آنها': 738,
 'شده': 725,
 'مورد': 703,
 '"': 700,
 'تا': 694,
 'نيز': 680,
 'شد': 627,
 'دو': 610,
 'قرار': 604,
 'بود': 601,
 'او': 586,
 'دارد': 585,
 'صورت': 566,
 'نظر': 562,
 'سال': 556,
 'باشد': 550,
 '2': 524,
 'ديگر': 499,
 'كار': 492,
 'هم': 491,
 'گروه': 487,
 '1': 472,
 'ميكند': 464,
 'وجود': 450,
 'بين': 441,
 'بايد': 435,
 'استفاده': 434,
 'آموزش': 431,
 '/': 427,
 'كرد': 415,
 'ميباشد': 404,
 'توجه': 380,
 'زيروبمي': 380,
 'شود': 373,
 'انجام': 368,
 'آب': 358,
 'اگر': 356,
 'بعد': 355,
 'هستند': 349,
 '3': 347,
 'نشان': 334,
 'يعني': 319,
 'عنوان': 313,
 'طور': 311,
 'روي': 310,
 'سطح': 296,
 'درصد': 289,
 'اجتماعي': 289,
 'ولي': 288,
 'نفت': 287,
 'زير': 284,
 'ميد

In [39]:
new_word_count = dict( sorted(new_word_count.items(), key=operator.itemgetter(1),reverse=True))

In [41]:
new_word_count

{'و': 11775,
 'UNK': 8732,
 '.': 8722,
 'در': 8471,
 'به': 7206,
 '،': 7158,
 'از': 6163,
 'كه': 5507,
 'است': 4210,
 'اين': 3708,
 'را': 3495,
 'با': 2671,
 '-': 1745,
 '(': 1694,
 ':': 1690,
 'آن': 1607,
 ')': 1436,
 'براي': 1350,
 'يك': 1253,
 'بر': 1176,
 'ميشود': 1017,
 '#': 884,
 'خود': 828,
 'يا': 794,
 'هر': 746,
 'آنها': 738,
 'شده': 725,
 'مورد': 703,
 '"': 700,
 'تا': 694,
 'نيز': 680,
 'شد': 627,
 'دو': 610,
 'قرار': 604,
 'بود': 601,
 'او': 586,
 'دارد': 585,
 'صورت': 566,
 'نظر': 562,
 'سال': 556,
 'باشد': 550,
 '2': 524,
 'ديگر': 499,
 'كار': 492,
 'هم': 491,
 'گروه': 487,
 '1': 472,
 'ميكند': 464,
 'وجود': 450,
 'بين': 441,
 'بايد': 435,
 'استفاده': 434,
 'آموزش': 431,
 '/': 427,
 'كرد': 415,
 'ميباشد': 404,
 'توجه': 380,
 'زيروبمي': 380,
 'شود': 373,
 'انجام': 368,
 'آب': 358,
 'اگر': 356,
 'بعد': 355,
 'هستند': 349,
 '3': 347,
 'نشان': 334,
 'يعني': 319,
 'عنوان': 313,
 'طور': 311,
 'روي': 310,
 'سطح': 296,
 'درصد': 289,
 'اجتماعي': 289,
 'ولي': 288,
 'نفت': 287,
 'زي

In [105]:
how to perform on test dataset? separatly
using lexicon and UNK

SyntaxError: invalid syntax (584844077.py, line 1)

In [62]:
word_given_tag_count

{('و', '<S>'): 0,
 ('و', 'ADJ'): 0,
 ('و', 'ADV'): 0,
 ('و', 'AR'): 97,
 ('و', 'CON'): 11671,
 ('و', 'DEFAULT'): 0,
 ('و', 'DELM'): 0,
 ('و', 'DET'): 0,
 ('و', 'IF'): 0,
 ('و', 'INT'): 0,
 ('و', 'MORP'): 0,
 ('و', 'MQUA'): 0,
 ('و', 'MS'): 0,
 ('و', 'N'): 7,
 ('و', 'NP'): 0,
 ('و', 'OH'): 0,
 ('و', 'OHH'): 0,
 ('و', 'P'): 0,
 ('و', 'PP'): 0,
 ('و', 'PRO'): 0,
 ('و', 'PS'): 0,
 ('و', 'QUA'): 0,
 ('و', 'SPEC'): 0,
 ('و', 'V'): 0,
 ('UNK', '<S>'): 0,
 ('UNK', 'ADJ'): 1690,
 ('UNK', 'ADV'): 138,
 ('UNK', 'AR'): 478,
 ('UNK', 'CON'): 73,
 ('UNK', 'DEFAULT'): 4,
 ('UNK', 'DELM'): 7,
 ('UNK', 'DET'): 5,
 ('UNK', 'IF'): 1,
 ('UNK', 'INT'): 1,
 ('UNK', 'MORP'): 10,
 ('UNK', 'MQUA'): 2,
 ('UNK', 'MS'): 22,
 ('UNK', 'N'): 5503,
 ('UNK', 'NP'): 1,
 ('UNK', 'OH'): 1,
 ('UNK', 'OHH'): 2,
 ('UNK', 'P'): 16,
 ('UNK', 'PP'): 7,
 ('UNK', 'PRO'): 10,
 ('UNK', 'PS'): 2,
 ('UNK', 'QUA'): 8,
 ('UNK', 'SPEC'): 1,
 ('UNK', 'V'): 750,
 ('.', '<S>'): 0,
 ('.', 'ADJ'): 0,
 ('.', 'ADV'): 0,
 ('.', 'AR'): 0,
 ('.'

In [56]:
count_vocab = len(new_word_count)
tags = len(all_tags)
print(f'count vocab : {count_vocab}')
print(f'tags len : {tags}')

count vocab : 10218
tags len : 24


# observation matrix

In [ ]:
def create_observation_matrix(df, vocab_counts, alpha):
    vocabs = sorted(vocab_counts.keys())
    word_given_tag_count = {}
    for i in vocabs: # change name to vocab
        for j in all_tags:
                key = (i, j)
                word_given_tag_count[key] = 0

    for i in range(df.shape[0]):
        word, tag = df.word[i], df.tag[i]
        key = (word, tag)
        if word == " ": # for blank llines
            pass
        elif word not in vocabs:
            key = ("UNK", tag)
            word_given_tag_count[key] += 1
        else:
            word_given_tag_count[key] += 1  

    tag_count = df.tag.value_counts().to_dict()
    all_tags = sorted(tag_count.keys())
    observation_matrix = np.zeros((len(vocabs), len(all_tags))) 

    for i in range(observation_matrix.shape[0]):
        for j in range(observation_matrix.shape[1]):
            count = 0
            key = (vocabs[i], all_tags[j])
            if key in word_given_tag_count:
                count = word_given_tag_count[key]
            count_tag = tag_count[all_tags[j]]
            observation_matrix[i][j] = (count + alpha) / (count_tag + alpha*len(vocabs))   
    observation_matrix = np.delete(observation_matrix, 0, axis=1) 


            
                       
              

In [57]:
# word_given_tag_count = {}
# for i in all_words: # change name to vocab
#     for j in all_tags:
#             key = (i, j)
#             word_given_tag_count[key] = 0

# for i in range(df.shape[0]):
#     word, tag = df.word[i], df.tag[i]
#     key = (word, tag)
#     if word == " ": # for blank llines
#         pass
#     elif word not in all_words:
#         key = ("UNK", tag)
#         word_given_tag_count[key] += 1
#     else:
#         word_given_tag_count[key] += 1

In [58]:
len(word_given_tag_count)

245232

In [63]:
all_words = sorted(new_word_count.keys())
all_words

['!',
 '"',
 '#',
 '%',
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '....',
 '.....',
 '.........',
 '...حمدا',
 '/',
 '0',
 '0/',
 '000',
 '001',
 '01',
 '02',
 '03',
 '031',
 '05',
 '06',
 '0731',
 '1',
 '1-',
 '1.',
 '1.5',
 '10',
 '100',
 '1000',
 '100000',
 '103',
 '104',
 '105',
 '107',
 '108',
 '109',
 '11',
 '110',
 '113',
 '114',
 '117',
 '12',
 '120',
 '1200',
 '122',
 '124',
 '1248',
 '125',
 '128',
 '13',
 '130',
 '1316',
 '1318',
 '132',
 '1324',
 '1328',
 '1337',
 '1342',
 '1345',
 '1347',
 '1349',
 '1350',
 '1352',
 '1353',
 '1356',
 '1358',
 '1360',
 '1362',
 '1365',
 '1366',
 '1367',
 '1368',
 '1369',
 '137',
 '1370',
 '1371',
 '1372',
 '1373',
 '1374',
 '1375',
 '1376',
 '1377',
 '1378',
 '1379',
 '139',
 '14',
 '140',
 '1400',
 '142',
 '143',
 '1430',
 '15',
 '150',
 '1500',
 '150000',
 '151',
 '158',
 '16',
 '160',
 '1600',
 '1631',
 '164',
 '165',
 '166',
 '17',
 '170000',
 '18',
 '180',
 '1863',
 '1880',
 '19',
 '1909',
 '1930',
 '1938',
 '1939',
 '1

In [65]:
len(tag_count)

24

In [67]:
len(all_tags)

24

In [68]:
all_words[0]

'!'

In [69]:
unique_tags = len(tag_count)  # except <S> tag
# unique_words = len(word_count)  # except empty word
all_tags = sorted(tag_count.keys())
all_words = sorted(new_word_count.keys())

observation_matrix = np.zeros((len(all_words), unique_tags)) 

for i in range(observation_matrix.shape[0]):
    for j in range(observation_matrix.shape[1]):
        count = 0
        key = (all_words[i], all_tags[j])
        if key in word_given_tag_count:
            count = word_given_tag_count[key]
        count_tag = tag_count[all_tags[j]]
        observation_matrix[i][j] = count / count_tag 
observation_matrix = np.delete(observation_matrix, 0, axis=1)               

In [76]:
word_given_tag_count

{('و', '<S>'): 0,
 ('و', 'ADJ'): 0,
 ('و', 'ADV'): 0,
 ('و', 'AR'): 97,
 ('و', 'CON'): 11671,
 ('و', 'DEFAULT'): 0,
 ('و', 'DELM'): 0,
 ('و', 'DET'): 0,
 ('و', 'IF'): 0,
 ('و', 'INT'): 0,
 ('و', 'MORP'): 0,
 ('و', 'MQUA'): 0,
 ('و', 'MS'): 0,
 ('و', 'N'): 7,
 ('و', 'NP'): 0,
 ('و', 'OH'): 0,
 ('و', 'OHH'): 0,
 ('و', 'P'): 0,
 ('و', 'PP'): 0,
 ('و', 'PRO'): 0,
 ('و', 'PS'): 0,
 ('و', 'QUA'): 0,
 ('و', 'SPEC'): 0,
 ('و', 'V'): 0,
 ('UNK', '<S>'): 0,
 ('UNK', 'ADJ'): 1690,
 ('UNK', 'ADV'): 138,
 ('UNK', 'AR'): 478,
 ('UNK', 'CON'): 73,
 ('UNK', 'DEFAULT'): 4,
 ('UNK', 'DELM'): 7,
 ('UNK', 'DET'): 5,
 ('UNK', 'IF'): 1,
 ('UNK', 'INT'): 1,
 ('UNK', 'MORP'): 10,
 ('UNK', 'MQUA'): 2,
 ('UNK', 'MS'): 22,
 ('UNK', 'N'): 5503,
 ('UNK', 'NP'): 1,
 ('UNK', 'OH'): 1,
 ('UNK', 'OHH'): 2,
 ('UNK', 'P'): 16,
 ('UNK', 'PP'): 7,
 ('UNK', 'PRO'): 10,
 ('UNK', 'PS'): 2,
 ('UNK', 'QUA'): 8,
 ('UNK', 'SPEC'): 1,
 ('UNK', 'V'): 750,
 ('.', '<S>'): 0,
 ('.', 'ADJ'): 0,
 ('.', 'ADV'): 0,
 ('.', 'AR'): 0,
 ('.'

In [79]:
all_words.index('2')

163

In [80]:
observation_matrix[163][13]

0.00479370597383588

In [78]:
all_tags[13]

'N'

In [70]:
observation_matrix.shape

(10218, 24)

In [82]:
diff = observation_matrix == np.zeros((len(all_words), unique_tags))
y = np.where(diff == False)[0].tolist()
y

[0,
 1,
 2,
 2,
 3,
 4,
 4,
 5,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,


In [83]:
len(y)

11514

In [85]:
_df = pd.DataFrame(observation_matrix, columns = list(all_tags), index=list(all_words))
display(_df)

,<S>,ADJ,ADV,AR,CON,DEFAULT,DELM,DET,IF,INT,...,NP,OH,OHH,P,PP,PRO,PS,QUA,SPEC,V
!,0.0,0.000000,0.0,0.0,0.0,0.00,0.002476,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
"""",0.0,0.000000,0.0,0.0,0.0,0.00,0.027086,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
#,0.0,0.000000,0.0,0.0,0.0,0.60,0.033741,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
%,0.0,0.000000,0.0,0.0,0.0,0.00,0.002670,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
(,0.0,0.000000,0.0,0.0,0.0,0.05,0.065508,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‏در,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000092,0.0,0.0,0.0,0.0,0.0,0.0
‏مربوط,0.0,0.000067,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
“,0.0,0.000000,0.0,0.0,0.0,0.00,0.000426,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
”,0.0,0.000000,0.0,0.0,0.0,0.00,0.000464,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
# np.save('/mnt/DAE855F7E855D1FD/github_msc/NLP/HW3/observation_matrix.npy', observation_matrix)

elimination for both martixes
smoothing
calculate pi
viterbi algo
test data????